In [1]:
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import geopandas as gpd
from scipy import stats
pd.set_option('display.max_columns', None)
from scipy.stats import skew, kurtosis
import pygris
from shapely.geometry import Polygon
import shapely
from inconsistency_utils import *

In [2]:
#Read the original dataset

df = pd.read_parquet("C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/FimaNfipClaims.parquet.gzip")

## Third Dataset [NA County]

In [3]:
df_geographic_unique = df[['state', 'reportedZipCode', 'countyCode', 'censusTract', 'censusBlockGroupFips', 'latitude', 'longitude', 'yearOfLoss']].drop_duplicates()

In [4]:
# Defining conditions
condition1 = (df_geographic_unique['latitude'].notna() & 
              df_geographic_unique['censusBlockGroupFips'].notna() & 
              df_geographic_unique['countyCode'].isna() & 
              df_geographic_unique['reportedZipCode'].notna())

# Filtering the dataframe based on the combined conditions
df_geographic_unique = df_geographic_unique[condition1]

In [5]:
df_geographic_unique.head()

,state,reportedZipCode,countyCode,censusTract,censusBlockGroupFips,latitude,longitude,yearOfLoss
21824,FL,25302.0,NaN,5.403900e+10,5.403900e+11,38.3,-81.6,2004
23323,VA,24112.0,NaN,5.169000e+10,5.169000e+11,36.7,-79.8,2018
70859,AL,35766.0,NaN,7.212700e+10,7.212700e+11,18.4,-66.0,1990
81466,CA,95901.0,NaN,2.011706e+10,2.011706e+11,39.8,-96.6,1997
92161,NC,28401.0,NaN,2.404795e+10,2.404795e+11,38.4,-75.1,1996


In [6]:
print(sum(df_geographic_unique['state'].isna()))
print(sum(df_geographic_unique['reportedZipCode'].isna()))
print(sum(df_geographic_unique['countyCode'].isna()))
print(sum(df_geographic_unique['censusTract'].isna()))
print(sum(df_geographic_unique['censusBlockGroupFips'].isna()))
print(sum(df_geographic_unique['latitude'].isna()))
print(sum(df_geographic_unique['yearOfLoss'].isna()))

0
0
4964
0
0
0
0


In [7]:
#convert units to string type to easily map

columns_to_format = {
    'reportedZipCode': 5,
    'censusBlockGroupFips': 12,
    'censusTract': 11
}

df_geographic_unique = format_geographic_units(df_geographic_unique, columns_to_format)

In [8]:
# Creating year bins to map (NEED TO UPDATE IT ONCE WE HAVE 2023 ZIPCODE SHAPEFILE DATA AND THE LATEST ORIGINAL FEMA DATASET) 

bins_1980_2021 = [df['yearOfLoss'].min(), 1980, 1990, 2000, 2010, 2020, df['yearOfLoss'].max() + 1]
labels_1980_2021 = [0, 1980, 1990, 2000, 2010, 2020]

custom_bins = [0, 2000] + list(range(2010, 2023)) + [2024]
custom_labels = [0, 2000] + list(range(2010, 2023))

df_geographic_unique = process_year_of_loss(df_geographic_unique, bins_1980_2021, labels_1980_2021, custom_bins, custom_labels)

In [9]:
df_geographic_unique.head()

,state,reportedZipCode,countyCode,censusTract,censusBlockGroupFips,latitude,longitude,yearOfLoss,yearOfLoss_1980_2021,zip_year_bin
21824,FL,25302,NaN,54039001100,540390011006,38.3,-81.6,2004,2000,2000
23323,VA,24112,NaN,51690000500,516900005003,36.7,-79.8,2018,2010,2018
70859,AL,35766,NaN,72127004600,721270046004,18.4,-66.0,1990,1990,0
81466,CA,95901,NaN,20117060510,201170605103,39.8,-96.6,1997,1990,0
92161,NC,28401,NaN,24047950300,240479503002,38.4,-75.1,1996,1990,0


## Read shapefiles

In [10]:
units = ['state', 'lat_long', 'BG', 'zipcode', 'Tract']
base_path = "C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/"
df_shapefiles = load_and_rename_geographic_data_whole(units, base_path)

state_df = df_shapefiles['state']
lat_long_df = df_shapefiles['lat_long']
BG_df = df_shapefiles['BG']
zipcode_df = df_shapefiles['zipcode']
Tract_df = df_shapefiles['Tract']

Using the default year of 2021


C:\Users\Asus\Box\Home Folder lk220\Private\Duke\Duke Courses\Research\FloodDamagePrediction\Code\inconsistency_utils.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_df.rename(columns={'geometry': 'geometry_state'}, inplace=True)


ArrowMemoryError: realloc of size 1289748480 failed

## Geometry Intersection creation

In [ ]:
# Filter for post 1980

df_geographic_unique = df_geographic_unique[df_geographic_unique['yearOfLoss_1980_2021']!=0]

In [ ]:
# Setting the multi-index on lat_long_df
lat_long_df.set_index(['latitude', 'longitude'], inplace=True)

# Mapping the values
df_geographic_unique['geometry_lat_long'] = df_geographic_unique.set_index(['latitude', 'longitude']).index.map(lat_long_df['geometry_lat_long'])

# Resetting the index of lat_long_df (return to multi-index)
lat_long_df.reset_index(inplace=True)

In [ ]:
# Initial mapping with multi-index
BG_df.set_index(['GEOID'], inplace=True)
df_geographic_unique['geometry_BG'] = df_geographic_unique.set_index(['censusBlockGroupFips']).index.map(BG_df['geometry_BG'])

BG_df.reset_index(inplace=True)

In [ ]:
# Initial mapping with multi-index
zipcode_df.set_index(['ZIPcode', 'year'], inplace=True)
df_geographic_unique['geometry_zipcode'] = df_geographic_unique.set_index(['reportedZipCode', 'zip_year_bin']).index.map(zipcode_df['geometry_zipcode'])

zipcode_df.reset_index(inplace=True)

In [ ]:
# Setting the multi-index on state_df
state_df.set_index(['STUSPS'], inplace=True)

# Mapping the values
df_geographic_unique['geometry_state'] = df_geographic_unique.set_index(['state']).index.map(state_df['geometry_state'])

state_df.reset_index(inplace=True)

In [ ]:
# Initial mapping with multi-index
Tract_df.set_index(['censusTractID'], inplace=True)
df_geographic_unique['geometry_tract'] = df_geographic_unique.set_index(['censusTract']).index.map(Tract_df['geometry_tract'])

# Resetting the index of Tract_df (return to multi-index)
Tract_df.reset_index(inplace=True)

#### Drop rows with missing shapefiles

In [ ]:
df_geographic_unique = df_geographic_unique[(df_geographic_unique['geometry_BG'].notna())
                              & (df_geographic_unique['geometry_zipcode'].notna())
                              & (df_geographic_unique['geometry_tract'].notna())]

#### Creating the intersections

In [ ]:
# Create an empty GeoDataFrame to store the intersection results
new_unit_df = gpd.GeoDataFrame(columns=['reportedZipCode', 'countyCode', 'censusTract',
                                       'censusBlockGroupFips', 'latitude', 'longitude', 'year', 'year_zipcode', 'state', 'geometry_zipcode',
                                        'geometry_tract','geometry_BG','geometry_lat_long','geometry_state',
                                         'cbgInconsistent', 'tractInconsistent', 'countyInconsistent', 'stateInconsistent', 'latlongInconsistent', 'multiple', 'noOverlap', 'oneWrong'])

for idx_unit, row_unit in df_geographic_unique.iterrows():
    year = row_unit['yearOfLoss_1980_2021']
    year_zipcode = row_unit['zip_year_bin']
    bg_id = row_unit['censusBlockGroupFips']
    bg_geometry = row_unit['geometry_BG']
    tract_id = row_unit['censusTract']
    tract_geometry = row_unit['geometry_tract']
    county_id = row_unit['countyCode']
    state = row_unit['state']
    state_geometry = row_unit['geometry_state']
    lat_long_geometry = row_unit['geometry_lat_long']
    lat = row_unit['latitude']
    long = row_unit['longitude']
    zipcode_geometry = row_unit['geometry_zipcode']
    zipcode = row_unit['reportedZipCode']
    
    # Compute intersection geometry
    #intersection_geometry = bg_geometry.intersection(lat_long_geometry).intersection(zipcode_geometry).intersection(county_geometry).intersection(state_geometry).intersection(tract_geometry)
    
    # First intersection
    intersection_1 = bg_geometry.intersection(lat_long_geometry)

    # Second intersection
    intersection_2 = intersection_1.intersection(zipcode_geometry)

    
    # Third intersection
    # Fourth intersection
    intersection_4 = intersection_2.intersection(tract_geometry)

    # Fifth intersection
    intersection_geometry = intersection_4.intersection(state_geometry)
    if intersection_geometry is None or intersection_geometry.is_empty:
        new_unit_df = pd.concat([new_unit_df, pd.DataFrame({
            'reportedZipCode': [zipcode],
            'countyCode': [county_id],
            'censusTract': [tract_id],
            'censusBlockGroupFips': [bg_id],
            'latitude': [lat],
            'longitude': [long],
            'year': [year],
            'year_zipcode': [year_zipcode],
            'state': [state],
            'geometry_zipcode': [zipcode_geometry],
            'geometry_tract': [tract_geometry],
            'geometry_BG': [bg_geometry],
            'geometry_lat_long': [lat_long_geometry],
            'geometry_state': [state_geometry]
        })], ignore_index=True)

In [ ]:
len(new_unit_df)

In [ ]:
for idx_unit, row_unit in new_unit_df.iterrows():
    year = row_unit['year']
    bg_id = row_unit['censusBlockGroupFips']
    bg_geometry = row_unit['geometry_BG']
    tract_id = row_unit['censusTract']
    tract_geometry = row_unit['geometry_tract']
    county_id = row_unit['countyCode']
    state = row_unit['state']
    state_geometry = row_unit['geometry_state']
    lat_long_geometry = row_unit['geometry_lat_long']
    lat = row_unit['latitude']
    long = row_unit['longitude']
    zipcode_geometry = row_unit['geometry_zipcode']
    zipcode = row_unit['reportedZipCode']
    

    intersection_geometry = lat_long_geometry.intersection(tract_geometry).intersection(zipcode_geometry).intersection(state_geometry)

    if intersection_geometry is None or intersection_geometry.is_empty:
        new_unit_df.at[idx_unit, 'cbgInconsistent'] = 0
    else:
        new_unit_df.at[idx_unit, 'cbgInconsistent'] = 1   

In [ ]:
new_unit_df['cbgInconsistent'].sum()

In [ ]:
for idx_unit, row_unit in new_unit_df.iterrows():
    year = row_unit['year']
    bg_id = row_unit['censusBlockGroupFips']
    bg_geometry = row_unit['geometry_BG']
    tract_id = row_unit['censusTract']
    tract_geometry = row_unit['geometry_tract']
    county_id = row_unit['countyCode']
    state = row_unit['state']
    state_geometry = row_unit['geometry_state']
    lat_long_geometry = row_unit['geometry_lat_long']
    lat = row_unit['latitude']
    long = row_unit['longitude']
    zipcode_geometry = row_unit['geometry_zipcode']
    zipcode = row_unit['reportedZipCode']
    
    intersection_geometry = bg_geometry.intersection(zipcode_geometry).intersection(lat_long_geometry).intersection(state_geometry)

    if intersection_geometry is None or intersection_geometry.is_empty:
        new_unit_df.at[idx_unit,'tractInconsistent'] = 0
    else:
        new_unit_df.at[idx_unit,'tractInconsistent'] = 1

In [ ]:
new_unit_df['tractInconsistent'].sum()

In [ ]:
for idx_unit, row_unit in new_unit_df.iterrows():
    year = row_unit['year']
    bg_id = row_unit['censusBlockGroupFips']
    bg_geometry = row_unit['geometry_BG']
    tract_id = row_unit['censusTract']
    tract_geometry = row_unit['geometry_tract']
    county_id = row_unit['countyCode']
    state = row_unit['state']
    state_geometry = row_unit['geometry_state']
    lat_long_geometry = row_unit['geometry_lat_long']
    lat = row_unit['latitude']
    long = row_unit['longitude']
    zipcode_geometry = row_unit['geometry_zipcode']
    zipcode = row_unit['reportedZipCode']
    
    intersection_geometry = bg_geometry.intersection(lat_long_geometry).intersection(zipcode_geometry).intersection(tract_geometry).intersection(state_geometry)

    if intersection_geometry is None or intersection_geometry.is_empty:
        new_unit_df.at[idx_unit,'countyInconsistent'] = 0
    else:
        new_unit_df.at[idx_unit,'countyInconsistent'] = 1

In [ ]:
new_unit_df['countyInconsistent'].sum()

In [ ]:
for idx_unit, row_unit in new_unit_df.iterrows():
    year = row_unit['year']
    bg_id = row_unit['censusBlockGroupFips']
    bg_geometry = row_unit['geometry_BG']
    tract_id = row_unit['censusTract']
    tract_geometry = row_unit['geometry_tract']
    county_id = row_unit['countyCode']
    state = row_unit['state']
    state_geometry = row_unit['geometry_state']
    lat_long_geometry = row_unit['geometry_lat_long']
    lat = row_unit['latitude']
    long = row_unit['longitude']
    zipcode_geometry = row_unit['geometry_zipcode']
    zipcode = row_unit['reportedZipCode']

    intersection_geometry = bg_geometry.intersection(zipcode_geometry).intersection(tract_geometry).intersection(lat_long_geometry)

    if intersection_geometry is None or intersection_geometry.is_empty:
        new_unit_df.at[idx_unit,'stateInconsistent'] = 0
    else:
        new_unit_df.at[idx_unit,'stateInconsistent'] = 1

In [ ]:
new_unit_df['stateInconsistent'].sum()

In [ ]:
for idx_unit, row_unit in new_unit_df.iterrows():
    year = row_unit['year']
    bg_id = row_unit['censusBlockGroupFips']
    bg_geometry = row_unit['geometry_BG']
    tract_id = row_unit['censusTract']
    tract_geometry = row_unit['geometry_tract']
    county_id = row_unit['countyCode']
    state = row_unit['state']
    state_geometry = row_unit['geometry_state']
    lat_long_geometry = row_unit['geometry_lat_long']
    lat = row_unit['latitude']
    long = row_unit['longitude']
    zipcode_geometry = row_unit['geometry_zipcode']
    zipcode = row_unit['reportedZipCode']
    
    intersection_geometry = bg_geometry.intersection(zipcode_geometry).intersection(tract_geometry).intersection(state_geometry)

    if intersection_geometry is None or intersection_geometry.is_empty:
        new_unit_df.at[idx_unit,'latlongInconsistent'] = 0
    else:
        new_unit_df.at[idx_unit,'latlongInconsistent'] = 1

In [ ]:
new_unit_df['latlongInconsistent'].sum()

In [ ]:
for idx_unit, row_unit in new_unit_df.iterrows():
    year = row_unit['year']
    bg_id = row_unit['censusBlockGroupFips']
    bg_geometry = row_unit['geometry_BG']
    tract_id = row_unit['censusTract']
    tract_geometry = row_unit['geometry_tract']
    county_id = row_unit['countyCode']
    state = row_unit['state']
    state_geometry = row_unit['geometry_state']
    lat_long_geometry = row_unit['geometry_lat_long']
    lat = row_unit['latitude']
    long = row_unit['longitude']
    zipcode_geometry = row_unit['geometry_zipcode']
    zipcode = row_unit['reportedZipCode']
    
    intersection_geometry = bg_geometry.intersection(lat_long_geometry).intersection(tract_geometry).intersection(state_geometry)

    if intersection_geometry is None or intersection_geometry.is_empty:
        new_unit_df.at[idx_unit,'zipInconsistent'] = 0
    else:
        new_unit_df.at[idx_unit,'zipInconsistent'] = 1

In [ ]:
print(new_unit_df['zipInconsistent'].sum())

In [ ]:
mask = (new_unit_df['latlongInconsistent'] +  new_unit_df['zipInconsistent']+ new_unit_df['stateInconsistent'] + new_unit_df['countyInconsistent'] + new_unit_df['tractInconsistent'] + new_unit_df['cbgInconsistent']) > 1
new_unit_df.loc[mask, 'multiple'] = 1
new_unit_df.loc[~mask, 'multiple'] = 0

print(sum(new_unit_df['multiple']))

In [ ]:
condition = (new_unit_df['latlongInconsistent'] + 
             new_unit_df['stateInconsistent'] + 
             new_unit_df['countyInconsistent'] + 
             new_unit_df['tractInconsistent'] + 
             new_unit_df['cbgInconsistent'] +
             new_unit_df['zipInconsistent']) == 0

new_unit_df['noOverlap'] = 0  # Default to 0
new_unit_df.loc[condition, 'noOverlap'] = 1

print(sum(new_unit_df['noOverlap']))

In [ ]:
condition = (new_unit_df['latlongInconsistent'] + 
             new_unit_df['stateInconsistent'] + 
             new_unit_df['countyInconsistent'] + 
             new_unit_df['tractInconsistent'] + 
             new_unit_df['cbgInconsistent'] +
             new_unit_df['zipInconsistent']) == 1

new_unit_df['oneWrong'] = 0  # Default to 0
new_unit_df.loc[condition, 'oneWrong'] = 1

print(sum(new_unit_df['oneWrong']))


In [ ]:
# Initialize the count dictionary for 'Wrong' values
wrong_counts = {
    'lat_long_geometry': 0,
    'state_geometry': 0,
    'county_geometry': 0,
    'tract_geometry': 0,
    'bg_geometry': 0,
    'zip_geometry': 0  # added for zip codes
}

# Iterate through the DataFrame
for idx_unit, row_unit in new_unit_df.iterrows():
    if row_unit['oneWrong'] == 1:
        if row_unit['latlongInconsistent'] == 1:
            new_unit_df.at[idx_unit, 'lat_long_geometry'] = 'Wrong'
            wrong_counts['lat_long_geometry'] += 1
        if row_unit['stateInconsistent'] == 1:
            new_unit_df.at[idx_unit, 'state_geometry'] = 'Wrong'
            wrong_counts['state_geometry'] += 1
        if row_unit['tractInconsistent'] == 1:
            new_unit_df.at[idx_unit, 'tract_geometry'] = 'Wrong'
            wrong_counts['tract_geometry'] += 1
        if row_unit['cbgInconsistent'] == 1:
            new_unit_df.at[idx_unit, 'bg_geometry'] = 'Wrong'
            wrong_counts['bg_geometry'] += 1
        if row_unit['zipInconsistent'] == 1:  # added for zip codes
            new_unit_df.at[idx_unit, 'zip_geometry'] = 'Wrong'
            wrong_counts['zip_geometry'] += 1

# Print the 'Wrong' counts
for category, count in wrong_counts.items():
    print(f"{category}: {count} 'Wrong' values")


In [ ]:
new_unit_df['CBG_Zip_Mutually_Inconsistent'] = np.where(
    (new_unit_df['latlongInconsistent'] == 0) & 
    (new_unit_df['cbgInconsistent'] == 1) & 
    (new_unit_df['zipInconsistent'] == 1), 1, 0)

In [ ]:
sum(new_unit_df['CBG_Zip_Mutually_Inconsistent'] == 1)

In [ ]:
len(new_unit_df[(new_unit_df['latlongInconsistent'] == 1) &(new_unit_df['cbgInconsistent'] == 1) & ((new_unit_df['latlongInconsistent'] + 
             new_unit_df['stateInconsistent'] + 
             new_unit_df['countyInconsistent'] + 
             new_unit_df['tractInconsistent'] + 
             new_unit_df['cbgInconsistent'] +
             new_unit_df['zipInconsistent'])  == 2)])

In [ ]:
columns_to_drop = [
    'geometry_tract', 'geometry_BG', 'geometry_zipcode', 'geometry_state', 'geometry_lat_long'
]

# Drop the columns
new_unit_df = new_unit_df.drop(columns= columns_to_drop)

In [ ]:
# Convert 'zipInconsistent' column to integer type.
new_unit_df['zipInconsistent'] = new_unit_df['zipInconsistent'].astype(int)

new_unit_df= new_unit_df.drop_duplicates()

new_unit_df.to_parquet('C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/inconsistency_dataframe_3.parquet.gzip', compression = 'gzip',  index=False)